Control Panel\Hardware and Sound\Devices and Printers

In [1]:
import serial
ser=serial.Serial("COM7",9600,timeout=1)
ser.write(bytes(f"{1},{1}\n", 'utf-8'))

4

In [2]:
import cv2 as cv
from cv2 import aruco
import numpy as np

calib_data_path = "./MultiMatrix.npz"
calib_data = np.load(calib_data_path)
print(calib_data.files)

cam_mat = calib_data["camMatrix"]
dist_coef = calib_data["distCoef"]
r_vectors = calib_data["rVector"]
t_vectors = calib_data["tVector"]

MARKER_SIZE = 8  # centimeters
marker_dict = aruco.Dictionary_get(aruco.DICT_4X4_50)
param_markers = aruco.DetectorParameters_create()

['camMatrix', 'distCoef', 'rVector', 'tVector']


In [ ]:
cap = cv.VideoCapture(1)
while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    marker_corners, marker_IDs, reject = aruco.detectMarkers(
        gray_frame, marker_dict, parameters=param_markers
    )
    if marker_corners:
        rVec, tVec, _ = aruco.estimatePoseSingleMarkers(
            marker_corners, MARKER_SIZE, cam_mat, dist_coef
        )
        total_markers = range(0, marker_IDs.size)
        for ids, corners, i in zip(marker_IDs, marker_corners, total_markers):
            cv.polylines(
                frame, [corners.astype(np.int32)], True, (0, 255, 255), 4, cv.LINE_AA
            )
            corners = corners.reshape(4, 2)
            corners = corners.astype(int)
            top_right = corners[0].ravel()
            top_left = corners[1].ravel()
            bottom_right = corners[2].ravel()
            bottom_left = corners[3].ravel()

            # Since there was mistake in calculating the distance approach point-outed in the Video Tutorial's comment
            # so I have rectified that mistake, I have test that out it increase the accuracy overall.
            # Calculating the distance
            distance = np.sqrt(
                tVec[i][0][2] ** 2 + tVec[i][0][0] ** 2 + tVec[i][0][1] ** 2
            )
            
            # Printing coordinate of distanc
            x=tVec[i][0][0]-(tVec[i][0][2]*0.087+5)
            y = tVec[i][0][2]
            
            if(ids[0]==1):
                coordinate = str(int(x))+","+str(int(y))
                ser.write(bytes(f"{int(x)},{int(y)}\n", 'utf-8'))
                print(coordinate)

            
            # Draw the pose of the marker
            point = cv.drawFrameAxes(frame, cam_mat, dist_coef, rVec[i], tVec[i], 4, 4)
            cv.putText(
                frame,
                f"id: {ids[0]} Dist: {round(distance, 2)}",
                top_right,
                cv.FONT_HERSHEY_PLAIN,
                1.3,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )
            cv.putText(
                frame,
                f"x:{int(x)} y: {int(y)} ",
                bottom_right,
                cv.FONT_HERSHEY_PLAIN,
                1.0,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )
            # print(ids, "  ", corners)
    cv.imshow("frame", frame)
    key = cv.waitKey(1)
    if key == ord("q"):
        break
cap.release()
cv.destroyAllWindows()

2,136
3,141
3,142
3,141
4,143
4,145
4,147
5,148
5,150
5,150
5,152
5,155
5,155
5,155
6,157
7,158
7,158
8,162
9,164
9,166
10,173
10,172
11,182
11,177
10,177
10,177
10,177
10,181
10,181
10,181
10,183
9,179
9,179
9,181
9,181
9,181
8,182
8,183
7,182
6,179
6,182
6,181
5,179
5,178
4,181
4,179
3,182
3,182
2,181
2,182
2,182
2,183
2,183
3,181
3,181
3,174
4,180
4,181
4,182
4,180
4,184
4,184
3,183
2,189
2,195
2,195
2,191
2,190
2,190
2,183
2,184
2,186
2,183
3,185
3,186
3,184
3,186
3,186
3,186
3,185
3,185
2,187
1,182
1,182
1,182
0,184
0,186
0,187
0,184
0,185
-1,184
-1,186
-1,186
-1,184
-1,184
-2,182
-2,183
-2,182
-2,184
-2,184
-1,184
-1,183
-1,181
-1,182
-1,180
-1,182
0,181
0,181
0,179
0,183
0,182
0,179
0,181
0,181
1,181
1,181
1,179
2,178
2,180
2,179
2,182
2,181
2,181
2,179
2,181
2,179
2,179
3,179
3,177
3,177
3,178
3,175
3,178
3,175
4,180
4,181
4,181
4,179
4,180
4,180
4,180
4,178
4,178
4,178
4,179
4,178
4,177
3,178
3,179
3,178
3,178
3,178
3,177
3,176
3,178
3,178
3,174
3,175
3,173
3,175
3,175
3,175
4

8,134
8,134
7,117
7,117
6,109
6,106
6,106
5,104
1,198
1,198
1,194
1,197
1,199
1,199
0,197
0,201
0,201
1,197
1,192
1,188
1,188
1,188
0,172
0,172
2,175
2,173
2,165
2,165
3,165
2,150
2,150
1,153
2,151
2,151
3,140
3,137
3,127
3,127
34,135
34,135
32,137
31,141
29,147
29,147
28,149
28,149
28,149
28,149
27,151
26,153
26,153
15,155
12,156
12,156
7,148
7,148
6,149
6,149
5,149
4,145
3,144
3,144
3,140
3,139
3,139
4,138
5,136
6,132
6,132
8,130
9,122
9,122
9,117
12,114
14,108
14,108
14,108
15,102
15,102
14,95
11,97
11,101
11,101
10,101
10,98
10,98
9,97
6,94
6,100
6,100
6,97
7,93
7,92
7,92
8,91
10,85
10,85
10,83
10,81
10,78
10,78
11,77
12,72
12,72
6,99
5,112
2,121
2,121
1,123
0,127
0,127
-3,133
-4,133
-6,134
-6,134
-9,136
-10,134
-10,134
-11,135
-12,134
-14,129
-14,129
-16,126
-19,122
-19,122
-19,123
-20,118
-22,115
-22,115
-24,111
-27,103
-27,103
-28,99
-29,96
-30,92
-30,92
-26,137
-26,137
-25,139
-25,139
-25,139
-25,139
-25,142
-24,141
-24,141
-24,141
-24,141
-24,141
-24,141
-13,143
-10,140
-10,14

44,157
44,157
43,158
44,161
43,161
42,162
41,161
41,161
39,157
35,164
35,164
29,165
28,162
28,162
26,163
25,165
25,166
21,154
23,143
23,134
25,127
25,127
25,122
26,117
26,117
57,193
55,193
54,194
57,203
57,203
53,194
53,195
52,193
52,193
51,197
44,203
37,207
34,209
34,209
28,212
25,211
25,211
23,210
21,211
17,201
15,197
15,197
14,194
14,189
14,189
15,186
16,182
16,179
16,179
18,160
20,159
22,157
22,157
22,147
22,137
23,131
23,131
23,131
26,118
26,118
26,110
4,130
4,130
3,135
3,137
2,142
2,142
1,151
1,152
1,152
0,168
0,165
0,165
0,164
0,167
0,167
2,170
3,174
5,169
5,169
9,166
9,166
9,166
14,166
33,136
33,136
34,135
45,163
45,163
44,165
43,172
43,172
42,172
42,177
38,190
34,203
34,203
13,217
13,217
11,215
11,217
11,206
11,206
12,206
10,208
10,208
9,195
9,195
9,195
10,195
11,185
11,185
12,184
12,175
15,164
15,164
31,130
31,130
6,136
6,139
6,139
6,144
5,147
4,153
4,153
3,160
2,163
2,163
2,165
1,174
0,173
0,173
0,169
0,164
0,164
0,164
2,170
4,171
4,171
6,169
11,162
11,162
14,160
21,171
24,1

-142,420
-142,420
-142,420
-142,420
-142,420
-142,420
-142,420
-144,425
-139,408
-139,408
-137,398
-143,411
-160,453
-140,406
-144,421
-144,421
-144,421
-142,417
-145,425
-145,425
-145,425
-140,411
-140,411
-140,411
-140,411
-140,411
-140,411
-140,411
-137,403
-137,403
-137,403
-140,411
-140,411
-142,417
-142,417
-142,417
-142,417
-142,417
-142,417
-142,417
-142,417
-142,417
-147,432
-147,432
-145,425
-142,417
-142,417
-142,417
-139,408
-139,407
-139,408
-142,417
-140,411
-140,411
-144,425
-138,406
-144,425
-144,425
-144,425
-141,416
-141,416
-143,424
-143,424
-139,411
-137,418
-134,422
-135,423
-134,418
-134,418
-131,406
-136,424
-136,424
-138,429
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-134,418
-134,418
-134,418
-134,418
-134,418
-134,418
-136,422
-136,422
-135,419
-135,419
-135,419
-135,419
-137,428
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-136,423
-

0,348
0,348
0,331
-2,336
-2,336
-3,338
-4,338
-7,355
-7,355
-8,350
-10,347
-10,347
-11,353
-14,365
-30,395
-30,395
-112,407
-112,407
-112,407
-109,391
-112,400
-115,414
-115,414
-115,412
-115,412
-115,412
-118,426
-116,417
-116,417
-116,419
-119,429
-116,419
-115,417
-115,417
-121,440
-118,429
-118,429
-118,429
-121,442
-121,442
-113,413
-114,418
-116,426
-116,426
-114,418
-114,418
-114,418
-114,418
-114,418
-116,428
-116,428
-116,428
-117,431
-114,418
-114,418
-114,418
-114,418
-113,413
-113,413
-118,431
-121,442
-121,442
-115,419
-115,418
-115,418
-115,418
-115,419
-118,431
-118,431
-118,431
-118,431
-118,431
-117,425
-117,425
-121,442
-117,427
-117,427
-117,427
-117,427
-117,427
-117,427
-117,427
-117,427
-117,427
-118,431
-118,431
-118,431
-118,431
-118,431
-117,427
-117,427
-117,427
-117,427
-117,427
-117,427
-118,432
-114,417
-118,431
-118,431
-120,440
-120,440
-120,440
-114,419
-114,419
-114,419
-116,427
-117,432
-120,442
-120,442
-116,429
-117,432
-117,432
-116,426
-116,429
-11

...

12,129
12,129
12,121
12,114
10,109
9,107
8,105
7,98
7,98
-15,93
-16,95
-21,100
-22,101
-25,102
-25,102
-26,99
-27,98
-27,98
-27,98
-26,95
-29,105
-29,105
-28,84
-29,85
-29,85
-28,77
-28,77
-28,80
-26,76
-26,76
-25,79
-25,79
-23,80
-23,80
-22,79
-13,111
47,297
63,399
63,399
62,375
60,361
60,361
31,391
31,391
31,391
31,380
34,407
34,407
34,407
34,400
34,400
30,413
30,425
30,425
32,442
27,388
32,385
32,385
29,359
31,362
30,349
30,349
27,357
27,357
27,352
25,350
25,345
25,345
24,339
22,346
18,335
17,328
16,327
22,305
22,294
22,294
23,298
21,291
21,291
22,298
21,297
15,299
4,288
0,294
-2,299
-2,299
-4,294
-12,270
-12,270
-17,272
-33,256
-33,256
-35,251
-44,234
-44,238
-55,233
-59,227
-59,227
-60,229
-61,231
-61,231
-58,239
-31,263
-31,257
-31,257
-32,263
-34,257
-34,257
-34,252
-28,252
-28,252
-27,249
-27,256
-25,252
-25,252
-24,252
-24,252
-24,252
-23,249
-23,248
-22,235
-22,235
-22,235
-22,235
-21,220
-21,220
-22,211
-32,208
-32,208
-32,199
-32,185
-33,177
-33,177
-34,179
-34,179
-34,182


-39,284
-40,288
-40,288
-40,293
-41,296
-41,294
-41,294
-43,302
-39,304
-35,313
-22,313
-21,317
-21,310
-15,328
-15,328
-13,320
-13,315
-13,315
-14,322
-14,314
-14,314
-13,317
-14,310
-12,315
-12,315
-12,315
-12,300
-10,294
-9,289
-9,289
-9,289
-8,295
-6,288
-6,288
-4,283
-4,283
-5,289
-3,289
0,277
0,282
0,282
0,284
2,284
4,282
4,282
4,266
4,274
4,274
4,284
7,257
10,262
10,262
10,262
13,254
15,249
15,249
15,249
21,245
23,249
23,249
24,243
27,239
26,240
25,232
23,231
24,239
25,239
45,284
45,284
46,289
46,289
47,289
47,289
47,289
48,300
44,301
-49,325
-49,325
-48,323
-48,323
-49,346
-47,338
-47,338
-47,338
-20,323
-16,316
-16,316
-16,316
-16,316
-12,316
-11,307
-11,307
-4,291
-4,291
-4,291
-3,287
-3,287
-3,264
-2,248
-1,248
-1,248
-3,238
-3,240
-1,236
-1,236
0,236
1,204
1,204
1,204
2,194
1,179
1,179
4,181
13,141
13,141
13,141
27,111
27,111
-12,121
-12,125
-12,125
-7,162
-3,164
-2,164
0,167
0,167
-2,168
-3,170
-3,170
-3,175
-3,172
-4,176
-4,176
-4,174
-3,179
-3,179
-3,177
-3,173
-2,171
-2

3,202
3,201
1,204
1,204
0,202
-3,193
-5,188
-5,188
-6,186
-5,178
-5,178
-5,178
-7,174
-9,164
-9,164
-9,157
-9,154
-9,154
-11,143
-12,139
-13,132
-13,132
-14,127
-15,118
-15,118
-15,118
-16,112
-16,112
-18,89
-17,80
-17,75
-17,75
-17,70
-16,66
-9,36
-7,38
-7,38
-5,39
-3,41
2,42
2,42
4,42
13,201
8,188
3,198
-1,192
-3,189
-5,187
-5,187
-7,182
-8,181
-8,181
-9,182
-9,180
-9,177
-9,177
-9,179
-8,179
-8,179
-7,182
-5,182
-3,189
-3,189
-2,187
-2,187
-3,184
-3,184
-3,185
-2,181
-2,181
-2,180
-3,184
-3,184
-3,184
-3,184
-2,175
-2,175
-2,177
-3,173
-5,180
-5,180
-6,182
-6,184
-6,184
-7,184
-11,187
-11,187
-11,183
-11,184
-11,184
-12,181
-13,188
-13,187
-13,187
-12,183
-12,183
-12,186
-12,191
-13,188
-13,188
-10,185
-10,185
-11,181
-11,181
-11,185
-11,185
-10,190
-10,192
-10,192
-10,190
-10,189
-9,193
-9,193
-7,192
-6,197
-6,197
-6,193
-6,194
-5,191
-5,191
-4,201
-4,196
-4,196
-4,201
-4,201
-2,200
-2,200
-1,200
0,203
0,203
0,202
-1,201
-1,201
-1,201
1,208
1,211
1,211
1,212
0,209
3,211
3,211
3,211

-34,116
-34,116
-34,115
-34,114
-34,114
-34,114
-33,110
-33,110
-33,110
-32,110
-30,104
-30,104
-29,102
-28,105
-25,95
-25,95
-23,86
-22,84
-22,84
-19,79
-19,78
-19,77
-17,69
-18,70
-17,67
-29,81
-28,83
-28,85
-28,85
-28,86
-28,86
-29,88
-27,89
-27,89
-27,89
21,95
21,95
21,93
21,95
19,97
19,97
19,97
17,104
17,104
17,104
15,105
8,110
8,110
3,114
0,114
0,114
-4,115
-7,112
-8,110
-8,110
-10,110
-11,108
-12,106
-12,106
-12,104
-15,99
-17,96
-17,96
-17,96
-20,94
-21,88
-21,88
-23,81
-23,78
-23,78
-23,78
-23,77
-23,77
-23,77
-23,76
-24,76
-24,76
-24,75
-22,75
-22,73
-24,84
-24,84
-25,91
-24,95
-24,95
-25,100
-25,102
-26,105
-25,117
-23,117
-21,123
-20,120
-20,120
-21,117
-20,120
-20,120
-20,120
-20,117
-19,119
-19,119
-19,119
-17,116
-17,116
-18,120
-19,119
-19,118
-19,118
-20,119
-22,111
-22,111
-23,112
-23,110
-25,112
-25,112
-25,107
-25,105
-25,105
-25,103
-25,101
-25,101
-25,101
-25,102
-25,98
-25,98
-25,98
-25,98
-23,90
-23,90
-23,86
-23,85
-23,85
-23,85
-23,85
-23,85
-23,82
-26,77
-26,

-29,141
-30,145
-30,145
-33,136
-36,134
-36,138
-36,138
-36,136
-36,134
-40,128
-51,156
-49,156
-49,156
-48,158
-48,158
-48,158
-48,158
-48,158
-48,158
-1,168
-2,166
-2,166
-1,165
0,161
0,160
0,160
0,155
0,152
0,152
0,150
0,149
0,149
-24,191
-25,191
-25,191
-30,195
-32,197
-32,197
-33,197
-33,192
-33,192
-31,193
-31,193
-31,195
-31,195
-32,192
-33,194
-33,194
-34,190
-34,181
-35,177
-35,177
-36,176
-37,173
-37,173
-38,167
-40,167
-39,158
-39,158
-40,152
-39,150
-38,150
-38,150
-37,149
-36,148
-36,148
-35,144
-34,133
-34,133
-34,135
-32,137
-32,137
-32,137
-28,133
-27,132
-27,134
-27,134
-24,135
-52,156
-51,160
-51,161
-51,164
-51,163
-51,163
-51,165
-49,169
-49,169
-50,169
-49,170
-41,175
-41,175
-41,175
-35,175
-35,175
-35,175
-35,175
-29,175
-29,175
-22,182
30,157
30,157
-3,192
-3,191
-4,190
-4,190
-5,194
-5,192
-5,192
-7,198
-8,198
-8,198
-8,194
-9,193
-9,193
-10,193
-11,187
-47,156
-48,157
-51,165
-51,165
-50,161
-51,163
-51,163
-52,163
0,181
0,181
-1,186
-43,189
-43,188
-42,189
-4

In [ ]:
cap.release()
cv.destroyAllWindows()